In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

import plotly.express as px

import os
import sys

from scipy.optimize import minimize
# code to manage where are lunch the file
if str(os.getcwd()).split("/")[-1] == "notebooks":
    sys.path.insert(1, os.path.realpath(os.path.pardir))
    path_b = "../"
else:
    path_b ='./'
import config as c

In [2]:
windows = c.windows
number_crypto = c.number_cryptos
market_cap = c.market_cap

df_name_index = pd.read_csv(f"{path_b}data/processed/first_{number_crypto}_crypto_list.csv", index_col=0)
df = pd.read_csv(f"{path_b}data/processed/market_cap_crypto.csv", index_col=0)
df['date'] = pd.to_datetime(df.index)
df['date'] = df['date'].dt.date
df.index = df['date']
del df['date']

df_close_price = pd.read_csv(f'{path_b}data/processed/close_price_crypto.csv', index_col=0)
df_close_price['date'] = pd.to_datetime(df_close_price.index)
df_close_price['date'] = df_close_price['date'].dt.date
df_close_price.index = df_close_price['date']
del df_close_price['date']

## use market cap file

df_market_cap = pd.read_csv(f'{path_b}data/processed/crypto_date_marketcap_sorted.csv', index_col=0)


df_market_cap_first_20 = df_market_cap.iloc[:number_crypto]

#select last date
first_date = df_market_cap_first_20['first_date_marketcap'].tail(1).values

first_date_final = pd.to_datetime(first_date[0])

df_final_data = df_close_price.drop(df_close_price[df_close_price.index < first_date_final].index)
df_final_data = df_final_data[df_market_cap_first_20['crypto_name']]

df_final_market_cap = df.drop(df[df.index < first_date_final].index)
df_final_market_cap = df_final_market_cap[df_market_cap_first_20['crypto_name']]

df_final_data_return = np.log(df_final_data/df_final_data.shift(1)).dropna()

AttributeError: module 'config' has no attribute 'number_cyptos'

In [ ]:
df_final_market_cap['sum'] = df_final_market_cap.sum(axis=1)
df_final_market_cap.head(4)

In [ ]:
columns = df_final_market_cap.columns[:-1]

df_weight_cap_weigthed= df_final_market_cap[columns]

for column in df_weight_cap_weigthed.columns:
    df_weight_cap_weigthed[column] = df_weight_cap_weigthed[column] / df_final_market_cap['sum']

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df_weight_cap_weigthed)
plt.legend(df_weight_cap_weigthed, loc="lower center", fancybox=True, shadow=True,ncol=4,bbox_to_anchor=(0.5, -0.25))
plt.title("Cap_weighted (Evolution of weights)")

In [ ]:
# voir pour comparer bitcoin avec les autres crypto (peut être agréger les cryptos)
fig = px.line(df_weight_cap_weigthed, x=df_weight_cap_weigthed.index, y=df_weight_cap_weigthed.columns, title="Cap_weighted (Evolution of weights)",labels={'value':'weight'})
fig.show()

In [ ]:
fig = px.scatter(df_weight_cap_weigthed['bitcoin'], x=df_weight_cap_weigthed['bitcoin'].index, y='bitcoin', title="Bitcoin (% of market cap)",labels={'bitcoin': "% of market cap"})
fig.show()

In [ ]:
print(df_final_data_return.shape)
print(df_weight_cap_weigthed.shape)
print(df_weight_cap_weigthed.shape[0] - df_final_data_return.shape[0])

diff = df_weight_cap_weigthed.shape[0] - df_final_data_return.shape[0]

In [ ]:
port_cap_weigthed = pd.DataFrame(np.sum(np.multiply(df_final_data_return, df_weight_cap_weigthed.iloc[diff:]),1)[2:],columns=['perf'])
port_cap_weigthed = port_cap_weigthed.drop(port_cap_weigthed[port_cap_weigthed.index == pd.to_datetime("2018-05-10") ].index)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot((port_cap_weigthed['perf']+1).cumprod()*100)
plt.legend(['Cap weigthed Index'],loc='upper left',fontsize='large',fancybox=True, shadow=True)
plt.title('Performance of Cap Weigthed Index')
plt.show()

In [ ]:
perf = (port_cap_weigthed['perf'].add(1)).cumprod()*100
fig = px.line(perf, x=perf.index, y=perf, title="Performance Cap Weigthed Index", labels={'y':'Performance (basis 100)'})
fig.show()

In [ ]:
df_test = pd.DataFrame((port_cap_weigthed+1).cumprod())
df_test.columns = ['cum_perf']
# df_test
df_test.drop(df_test[df_test.cum_perf != np.inf ].index)
#2018-05-09	-> bug à 2018-05-10 (inf)

In [ ]:
df_perf_test = pd.DataFrame(port_cap_weigthed)
df_perf_test.columns = ['perf']
final = df_perf_test.drop(df_perf_test[df_perf_test.index == pd.to_datetime("2018-05-10") ].index)

plt.plot((final['perf'].add(1).cumprod()*100))

CW = final['perf'].add(1).cumprod()*100
CW.to_csv(f"../data/processed/CW_{number_crypto}_price.csv")